In [37]:
from read_data import load_data_graphmetrics as load_data
%matplotlib inline
import matplotlib.pyplot as plt
import numpy
import itertools

d = load_data()

fields = ['Clustering_coeff', 'Assortativity', 'Assortativity_weighted', 'Clustering_coeff_weighted', 'Global_efficiency']

d_all = numpy.concatenate([d[freq][field][:, None]
                           for freq, field in itertools.product(d.keys(), fields)], axis=1)

print d_all.shape

session = d['alpha']['Session']
session_n = []

for i in session:                                                                                
    if i == 'rest':
        session_n.append(0)
    elif i == 'samatha':
        session_n.append(1)
    else:
        session_n.append(2)
        
session_n = numpy.array(session_n)

(36, 30)


In [114]:
import theano
from theano import tensor, function, shared, config, gradient

print config.floatX

class MLP():
    
    def __init__(self, layers=[30, 20, 20, 3]):
        
        self.parameters = []
        self.X = tensor.fmatrix()
        self.y = tensor.ivector()
        
        out = self.X
        in_size = layers[0]
        for i, l_size in enumerate(layers[1:]):
            scale = (2./in_size)**.5
            W_val = numpy.random.normal(scale=scale, size=(in_size, l_size))
            W_val = W_val.astype(config.floatX)
            W = shared(value=W_val)
            b_val = numpy.zeros(shape=(l_size, )).astype(config.floatX)
            b = shared(value=b_val)
            
            out = tensor.dot(out, W) + b
            if i == len(layers)-2:
                out = tensor.nnet.softmax(out)
            else:
                out = tensor.nnet.relu(out)
            
            self.parameters += [W, b]
            in_size = l_size
            
        prediction = out
        error = tensor.eq(tensor.argmax(prediction, axis=1), self.y).mean()
        cost = tensor.nnet.categorical_crossentropy(prediction, self.y).mean()
        
        self.lr = shared(value=numpy.float32(0.))
        updates = [(par, par - self.lr * gradient.grad(cost, par))
                  for par in self.parameters]
            
        self.train_fn = function(
            inputs=[self.X, self.y],
            outputs=[cost, error],
            updates=updates,
            allow_input_downcast=True)
        self.valid_fn = function(
            inputs=[self.X, self.y],
            outputs=[cost, error], allow_input_downcast=True)
            
        
    def train(self, X, y, X_val, y_val, lr=.01, n_epochs=2000):
        
        self.lr.set_value(numpy.float32(lr))
        scale = X.max(axis=0) - X.min(axis=0)
        min_val = X.min(axis=0)
        X_1 = (X + min_val)/scale
        X_val_1 = (X_val + min_val)/scale
        
        for epoch in range(n_epochs):
            train_out = self.train_fn(X_1, y)
            valid_out = self.valid_fn(X_val_1, y_val)
#             print epoch, 'train', train_out
#             print epoch, 'val  ', valid_out
            
            
        return train_out[1], valid_out[1]
        
        
        

float64


In [115]:
n_subjects = 12
val_error_sum = 0
train_error_sum = 0

for left_out in range(n_subjects):
    
    print 'subject left out: ', left_out
    valid_indices = [left_out, n_subjects+left_out, 2*n_subjects+left_out]
    data_train = numpy.delete(d_all, valid_indices, axis=0)
    data_valid = d_all[valid_indices, :]
    target_train = numpy.delete(session_n, valid_indices, axis=0)
    target_valid = session_n[valid_indices]
    
    mlp = MLP()
    train_error, val_error = mlp.train(data_train, target_train, data_valid, target_valid)
    print train_error, val_error
    val_error_sum += val_error
    train_error_sum += train_error
    
print train_error_sum/n_subjects, val_error_sum/n_subjects

subject left out:  0
0.69696969697 0.333333333333
subject left out:  1
0.69696969697 0.333333333333
subject left out:  2
0.757575757576 1.0
subject left out:  3
0.787878787879 0.333333333333
subject left out:  4
0.666666666667 0.333333333333
subject left out:  5
0.666666666667 0.333333333333
subject left out:  6
0.666666666667 0.666666666667
subject left out:  7
0.757575757576 0.333333333333
subject left out:  8
0.636363636364 0.333333333333
subject left out:  9
0.69696969697 0.666666666667
subject left out:  10
0.545454545455 0.666666666667
subject left out:  11
0.636363636364 0.333333333333
0.684343434343 0.472222222222
